# RealJoker


In [1]:
# Setup plotting
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
# Set Matplotlib defaults
plt.rc('figure', autolayout=True)
plt.rc('axes', labelweight='bold', labelsize='large',
       titleweight='bold', titlesize=18, titlepad=10)
plt.rc('animation', html='html5')


In [2]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer

joker_data = pd.read_csv('./input/joker.csv')

X = joker_data.copy()

features_num = [
    "date", "avg","idx",
]
features_cat = [
    "weekd", "n1", "n2",
    "n3", "n4","n5","j",
]

transformer_num = make_pipeline(
    SimpleImputer(strategy="constant"),  # there are a few missing values
    StandardScaler(),
)
transformer_cat = make_pipeline(
    SimpleImputer(strategy="constant", fill_value="NA"),
    OneHotEncoder(handle_unknown='ignore'),
)

preprocessor = make_column_transformer(
    (transformer_num, features_num),
    (transformer_cat, features_cat),
)

# stratify - make sure classes are evenlly represented across splits
X_train, X_valid, y_train, y_valid = \
    train_test_split(X, y, stratify=y, train_size=0.75)

X_train = preprocessor.fit_transform(X_train)
X_valid = preprocessor.transform(X_valid)

input_shape = [X_train.shape[1]]


NameError: name 'y' is not defined

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

# YOUR CODE HERE: define the model given in the diagram
model = keras.Sequential([
    layers.BatchNormalization(),
    layers.Dense(256, activation='relu', input_shape=input_shape),
    layers.BatchNormalization(),
    layers.Dropout(0.3),
    layers.Dense(256, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.3),
    layers.Dense(1, activation='sigmoid')
])


In [ ]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['binary_accuracy'],
)


In [ ]:
early_stopping = keras.callbacks.EarlyStopping(
    patience=5,
    min_delta=0.001,
    restore_best_weights=True,
)
history = model.fit(
    X_train, y_train,
    validation_data=(X_valid, y_valid),
    batch_size=512,
    epochs=200,
    callbacks=[early_stopping],
)

history_df = pd.DataFrame(history.history)
history_df.loc[:, ['loss', 'val_loss']].plot(title="Cross-entropy")
history_df.loc[:, ['binary_accuracy', 'val_binary_accuracy']].plot(
    title="Accuracy")


Epoch 1/200
175/175 [==============================] - 3s 14ms/step - loss: 0.4782 - binary_accuracy: 0.7730 - val_loss: 0.4334 - val_binary_accuracy: 0.8023
Epoch 2/200
175/175 [==============================] - 2s 11ms/step - loss: 0.4205 - binary_accuracy: 0.8035 - val_loss: 0.4094 - val_binary_accuracy: 0.8114
Epoch 3/200
175/175 [==============================] - 2s 14ms/step - loss: 0.4084 - binary_accuracy: 0.8090 - val_loss: 0.4001 - val_binary_accuracy: 0.8169
Epoch 4/200
175/175 [==============================] - 3s 17ms/step - loss: 0.4001 - binary_accuracy: 0.8137 - val_loss: 0.3928 - val_binary_accuracy: 0.8182
Epoch 5/200
175/175 [==============================] - 2s 13ms/step - loss: 0.3952 - binary_accuracy: 0.8164 - val_loss: 0.3887 - val_binary_accuracy: 0.8200
Epoch 6/200
175/175 [==============================] - 2s 11ms/step - loss: 0.3909 - binary_accuracy: 0.8182 - val_loss: 0.3928 - val_binary_accuracy: 0.8207
Epoch 7/200
175/175 [==============================]

In [ ]:
# Though we can see the training loss continuing to fall, the early stopping callback prevented 
# any overfitting. Moreover, the accuracy rose at the same rate as the cross-entropy fell, so it appears
#  that minimizing cross-entropy was a good stand-in. 
# All in all, it looks like this training was a success!
